In [0]:
# Import libraries 
#!pip install jsonlines
import time
import os
import os.path
import json
import jsonlines
import pickle
#!pip install sh
from sh import gunzip


In [0]:
#Mount Gdrive

#Gdrive: nlpclass90@gmail.com
#Pass: Hh@12345

#Gdrive: nlpclass94@gmail.com
#Pass: nlp12345@

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Set Working directory
os.chdir("/content/drive/My Drive/News_Summarization_with_BERT/newsroom_data")

In [0]:
# Untar the file 'complete.tar'; run completed, hence the commented command
#!tar -xvf complete.tar

release/
release/dev.jsonl.gz
release/test.jsonl.gz
release/train.jsonl.gz


In [0]:
# Unzip the .gz files; run completed, hence the commented commands 
#gunzip('train.jsonl.gz') #extracts .jsonl and deletes .gz
#gunzip('dev.jsonl.gz')
#gunzip('test.jsonl.gz')

In [0]:
#create new directory - DONE
cwd = os.getcwd()
print(cwd)
path = os.path.join(cwd, 'raw_data')

if not os.path.exists(path):
  os.mkdir(path)

/content/drive/My Drive/News_Summarization_with_BERT/newsroom_data


In [0]:
# Change directory to where stanford nlp is located
os.chdir("/content/drive/My Drive/News_Summarization_with_BERT/newsroom_data")



#unzip uploaded stanford corenlp folder ; runs completed hence, commented codes
#from zipfile import ZipFile

#with ZipFile('stanford-corenlp-full-2018-10-05.zip', 'r') as zipObj:
  # Extract all the contents of zip file in current directory
  #zipObj.extractall()

# Import the CORENLP module
!pip install stanfordnlp

from stanfordnlp.server import CoreNLPClient

     |████████████████████████████████| 163kB 2.8MB/s 


In [0]:
# Set the CORENLP_HOME environment variable to the location of the folder
os.environ['CORENLP_HOME'] = r"/content/drive/My Drive/News_Summarization_with_BERT/newsroom_data/stanford-corenlp-full-2018-10-05"

In [0]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
# 'client' splits the text input into words (tokens), and further splits the sequence of tokens into sentences. The output is returned as json object
client = CoreNLPClient(annotators=['tokenize','ssplit'], output_format= 'json', memory='4G', endpoint='http://localhost:9001')
print(client)

In [0]:
# Some codes from LEAFNATS(https://github.com/tshi04/LeafNATS/blob/master/LeafNATS/tools/newsroom_process/extract_corenlp.py) are borrowed and modified

def extract_data(jlfile, n):
  """Extracts all plain text from .jsonl file and writes article and respective summary text to individual files. 
  To delineate the summary from the main article, each sentence of the summary is prefixed with <s>.
  
  Keyword arguments:
  jlfile -- a jsonl file; each line in the file is an object representing a single article-summary pair
  n -- number of files to be extracted
  """
  sen_array = []

  # read the file entry (article) by entry (article)
  with jsonlines.open(jlfile) as f: 
    c = 1
    for entry in f:
      # print progress at every 5000th file
      if(c%5000 == 0): 
        print("%d files processed"%c)
      #print(entry.keys()) #keys are url, archive, title, date, text, summary, density...
      article = entry["text"]
      summary = entry["summary"]

      if article is None or summary is None:
        return ''

      article = article[:99999] # default max length string processed by corenlp = 100000
      # split text input into tokenized sentences containing tokenized words
      article = client.annotate(article)
      summary = client.annotate(summary)

      sen_arr = []
      # for every sentence
      for sen in article['sentences']:
        # return list of tokenized words
        sen = [s['originalText'] for s in sen['tokens'] if not '\n' in s['originalText']]
        # collapse the list into a proper sentence, with words separated by space
        sen = ' '.join(sen)
        # append sentence to the end of list 'sen_array'
        sen_arr.append(sen)
      # collapse the list into a proper article text, with sentences separated by space
      article = ' '.join(sen_arr)
    
      sen_arr = []
      # for every sentence
      for sen in summary['sentences']:
        # return list of tokenized words
        sen = [s['originalText'] for s in sen['tokens'] if not '\n' in s['originalText']]
        # add '<s>' as first element of the list
        sen = ['<s>']+sen
        # collapse the list into a proper sentence, with words separated by space
        sen = ' '.join(sen)
        # append sentence to the end of list 'sen_array'
        sen_arr.append(sen)
      # collapse the list into a proper summary text, with each sentence separated by space, but prefixed by '<s>'
      summary = ' '.join(sen_arr)

      # collect article and summary into a list
      sen_arr = [article, summary]
      # separate article text from summary text by new line tab
      sen_arr = '\n'.join(sen_arr)
      
      if 'train' in jlfile:
        # write article-summary text into new training file
        with open("./raw_data/newsroom_train_%d.p"%c, "w", newline='\n') as output:
          output.write(sen_arr)
          sen_array = []
      elif 'dev' in jlfile:
        # write article-summary text into new validation file
        with open("./raw_data/newsroom_valid_%d.p"%c, "w", newline='\n') as output:
          output.write(sen_arr)
          sen_array = []
      elif 'test' in jlfile:
        # write article-summary text into new test file
        with open("./raw_data/newsroom_test_%d.p"%c, "w", newline='\n') as output:
          output.write(sen_arr)
          sen_array = []


      if c >= n:
        print('%d files processed'%c)
        break

      c+=1

In [0]:
# Extract 60000 train, 20000 validation and 20000 test files
extract_data('train.jsonl', 60000)
extract_data('dev.jsonl', 20000)
extract_data('test.jsonl', 20000)

Starting server with command: java -Xmx4G -cp /content/drive/My Drive/News_Summarization_with_BERT/newsroom_data/stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-80899c6e9171474b.props -preload tokenize,ssplit
5000 files processed
10000 files processed
15000 files processed
20000 files processed
25000 files processed
30000 files processed
35000 files processed
40000 files processed
45000 files processed
50000 files processed
55000 files processed
60000 files processed
60000 files processed
5000 files processed
10000 files processed
15000 files processed
20000 files processed
20000 files processed
5000 files processed
10000 files processed
15000 files processed
20000 files processed
20000 files processed


In [0]:
# Check number of files in the raw_data directory
os.chdir("/content/drive/My Drive/News_Summarization_with_BERT/newsroom_data/raw_data")
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

100000
